<a href="https://colab.research.google.com/github/CSID-DGU/2023-1-CECD3-EagerBeavers-3/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import pandas as pd
import random
import numpy as np
import torch
import random
import numpy as np
import torch

!pip install torch_geometric
!pip install pytorch-tabnet

import torch
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np

!pip install torch

!pip install torch_geometric
!pip install pytorch-tabnet

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:

import torch
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np

class CPGPredictDataset(Dataset):
    def __init__(self, csv_file):
        super(CPGPredictDataset, self).__init__()

        self.df = pd.read_csv(csv_file)
        self.df.fillna(0, inplace=True)

        self.wo_cpgs_df = self.df.drop(columns=['index', 'nodes', 'edges', 'time'])
        self.wo_cpgs_df = torch.tensor(self.wo_cpgs_df.values.astype(np.float32))

        self.graph_list = self.cpgs2graph(self.df['nodes'], self.df['edges'])

        self.label_encoder = LabelEncoder()

        all_features = []
        for idx in range(len(self.graph_list)):
            graph_data = self.graph_list[idx]
            features = [data['feature'] for _, data in graph_data.nodes(data=True)]
            all_features.extend(features)

        self.label_encoder.fit(all_features)

    def __len__(self):
        return len(self.graph_list)

    def cpgs2graph_single(self, nodes, edges):
        # 노드 및 엣지 정보 추출
        nodes_info = [node.split(':') for node in nodes.split('|') if node]

        if isinstance(edges, str):
            edges_info = [edge.split('->') for edge in edges.split('|') if edge]
        elif isinstance(edges, int):
            edges_info = []
        else:
            raise ValueError("Invalid type for 'edges'. Should be either str or int.")

        # 그래프 생성
        G = nx.Graph()

        # 노드 추가
        for node_info in nodes_info:
            if len(node_info) > 1:  # 노드 정보가 제대로 있다면
                G.add_node(node_info[0], feature=node_info[1])

        # 엣지 추가
        for edge_info in edges_info:
            # 엣지 특성을 추가
            if len(edge_info) > 1:
                source, target_feature = edge_info[0], edge_info[1].split(':')
                if len(target_feature) > 1:
                    target, feature = target_feature[0], target_feature[1]
                    G.add_edge(source, target, feature=feature)
                else:
                    print("Error: Edge feature is missing.")
            else:
                print("Error: Incomplete edge information.")

        return G


    def cpgs2graph(self, nodes_list, edges_list):
        print('Convert "CPG"csv to graph')

        graph_list = []
        for nodes, edges in zip(nodes_list, edges_list):
            try:
                # 노드와 엣지를 그래프로 변환
                G = self.cpgs2graph_single(nodes, edges)
                graph_list.append(G)
            except Exception as e:
                print(f"Error creating graph: {e}")

        print('Complete!')
        return graph_list


    def __getitem__(self, idx):
        graph_data = self.graph_list[idx]

        nodes_info = [(node, self.label_encoder.transform([data['feature']])[0]) for node, data in graph_data.nodes(data=True)]
        edges_info = [(u, v, data['feature']) for u, v, data in graph_data.edges(data=True)]

        node_indices = [node[0] for node in nodes_info]
        edges = [(node_indices.index(u), node_indices.index(v)) for u, v, _ in edges_info]

        # edge_index 텐서 생성
        edge_index = torch.tensor(list(zip(*edges)), dtype=torch.long)

        # node feature 텐서 생성
        x = torch.tensor([node[1] for node in nodes_info], dtype=torch.float).view(-1, 1)

        data = Data(x=x, edge_index=edge_index)

        return data

In [ ]:
class CPGPredictDataset(Dataset):
    def __init__(self, csv_file):
        super(CPGPredictDataset, self).__init__()

        self.df = pd.read_csv(csv_file)
        self.df.fillna(0, inplace=True)

        self.wo_cpgs_df = self.df.drop(columns=['index', 'nodes', 'edges'])
        self.wo_cpgs_df = torch.tensor(self.wo_cpgs_df.values.astype(np.float32))

        self.graph_list = self.cpgs2graph(self.df['nodes'], self.df['edges'])

        self.label_encoder = LabelEncoder()

        all_features = []
        for idx in range(len(self.graph_list)):
            graph_data = self.graph_list[idx]
            features = [data['feature'] for _, data in graph_data.nodes(data=True)]
            all_features.extend(features)

        self.label_encoder.fit(all_features)

    def __len__(self):
        return len(self.graph_list)

    def cpgs2graph_single(self, nodes, edges):
        # 노드 및 엣지 정보 추출
        nodes_info = [node.split(':') for node in nodes.split('|') if node]

        if isinstance(edges, str):
            edges_info = [edge.split('->') for edge in edges.split('|') if edge]
        elif isinstance(edges, int):
            edges_info = []
        else:
            raise ValueError("Invalid type for 'edges'. Should be either str or int.")

        # 그래프 생성
        G = nx.Graph()

        # 노드 추가
        for node_info in nodes_info:
            if len(node_info) > 1:  # 노드 정보가 제대로 있다면
                G.add_node(node_info[0], feature=node_info[1])

        # 엣지 추가
        for edge_info in edges_info:
            # 엣지 특성을 추가
            if len(edge_info) > 1:
                source, target_feature = edge_info[0], edge_info[1].split(':')
                if len(target_feature) > 1:
                    target, feature = target_feature[0], target_feature[1]
                    G.add_edge(source, target, feature=feature)
                else:
                    print("Error: Edge feature is missing.")
            else:
                print("Error: Incomplete edge information.")

        return G


    def cpgs2graph(self, nodes_list, edges_list):
        print('Convert "CPG"csv to graph')

        graph_list = []
        for nodes, edges in zip(nodes_list, edges_list):
            try:
                # 노드와 엣지를 그래프로 변환
                G = self.cpgs2graph_single(nodes, edges)
                graph_list.append(G)
            except Exception as e:
                print(f"Error creating graph: {e}")

        print('Complete!')
        return graph_list


    def __getitem__(self, idx):
        graph_data = self.graph_list[idx]

        nodes_info = [(node, self.label_encoder.transform([data['feature']])[0]) for node, data in graph_data.nodes(data=True)]
        edges_info = [(u, v, data['feature']) for u, v, data in graph_data.edges(data=True)]

        node_indices = [node[0] for node in nodes_info]
        edges = [(node_indices.index(u), node_indices.index(v)) for u, v, _ in edges_info]

        # edge_index 텐서 생성
        edge_index = torch.tensor(list(zip(*edges)), dtype=torch.long)

        # node feature 텐서 생성
        x = torch.tensor([node[1] for node in nodes_info], dtype=torch.float).view(-1, 1)

        data = Data(x=x, edge_index=edge_index)

        return data


In [ ]:
num_node_features = 1  # 노드 피처의 차원


class GCN(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)

        x = global_mean_pool(x, data.batch)

        return x.view(-1)

#예측 결과 출력 함수
def interpret_prediction(prediction):
    # numpy.round를 사용하여 소수점 3자리에서 반올림
    rounded_pred = np.round(prediction, 3)

    # 소수점 이하 3자리까지만 출력하도록 문자열 형식으로 변환
    rounded_pred_str = f"{rounded_pred:.3f}"
    # 범위에 따른 문자열 반환
    # if 0 <= rounded_pred <= 0.5:
    #     return f"constant in Big-O Notation, predicted value was : {rounded_pred}"
    # elif 0.6 <= rounded_pred <= 1.5:
    #     return f"logn in Big-O Notation, predicted value was : {rounded_pred}"
    # elif 1.6 <= rounded_pred <= 2.5:
    #     return f"linear in Big-O Notation, predicted value was : {rounded_pred}"
    # elif 2.6 <= rounded_pred <= 3.5:
    #     return f"nlogn in Big-O Notation, predicted value was : {rounded_pred}"
    # elif 3.6 <= rounded_pred <= 4.5:
    #     return f"quadratic in Big-O Notation, predicted value was : {rounded_pred}"
    # else:
    #     return f"bigger than quadratic in Big-O Notation, predicted value was : {rounded_pred}"
    return float(rounded_pred_str)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CPG 생성

In [ ]:
import os

def execute_script(script_path):
    # 권한 부여하기
    os.system('chmod +x {}'.format(script_path))
    # 실행하기
    os.system('sh {}'.format(script_path))




# csv 생성 함수

In [ ]:
import os
import re

def process_all_dot_files(folder_path, output_csv_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".dot"):
            file_path = os.path.join(folder_path, file_name)
            print(f"Processing file: {file_name}")
            try:
                nodes_and_edges = extract_node_and_edge_info(file_path)
                write_to_csv(file_name, nodes_and_edges, output_csv_path)
            except IOError as e:
                print(f"Error processing file {file_name}: {e}")

def extract_node_and_edge_info(file_path):
    nodes = []
    edges = []

    with open(file_path, 'r') as file:
        dot_content = file.read()

        # Extract and print labels
        node_pattern = re.compile(r'"(\d+)"\s*\[label\s*=\s*<\s*\(([^,]+)')
        node_matches = node_pattern.findall(dot_content)

        for node_match in node_matches:
            node_number, literal_value = node_match
            node = f"{node_number}:{literal_value}"
            nodes.append(node)

        # Extract and print edge information
        edge_pattern = re.compile(r'"(\S+)"\s*->\s*"(\S+)"\s*\[\s*label\s*=\s*"([^"]+)"\s*\]')
        edge_matches = edge_pattern.findall(dot_content)

        for edge_match in edge_matches:
            source_node, target_node, label = edge_match
            cleaned_label = label.rstrip(':').strip()
            edge = f"{source_node}->{target_node}:{cleaned_label}"
            edges.append(edge)

    return '|'.join(nodes) + "$" + '|'.join(edges)

def write_to_csv_header(output_csv_path):
    with open(output_csv_path, 'w') as csv_file:
        # Write CSV header
        csv_file.write("index,nodes,edges,time\n")

def write_to_csv(file_name, nodes_and_edges, output_csv_path):
    cleaned_file_name = extract_numeric_part(file_name)

    # Split nodes and edges using "|"
    parts = nodes_and_edges.split("$")

    if len(parts) == 2:
        # Write file name, nodes, and edges to CSV
        with open(output_csv_path, 'a') as csv_file:
            csv_file.write(f"{cleaned_file_name},\"{parts[0]}\",\"{parts[1]}\"\n")
    else:
        # Handle the case when the format is unexpected
        with open(output_csv_path, 'a') as csv_file:
            csv_file.write("\"\",\"\",{nodes_and_edges}\n")

def extract_numeric_part(file_name):
    pattern = re.compile(r'\d+')
    match = pattern.search(file_name)

    if match:
        return match.group()

    return ""

In [ ]:
!pip install flask

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask_ngrok

In [ ]:
!pip install ngrok

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!cp ngrok /usr/local/bin

# Replace 'your-auth-token' with your actual ngrok authtoken
!ngrok authtoken 'your auth token'

--2023-12-16 16:50:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.28M  49.9MB/s    in 0.3s    

2023-12-16 16:50:33 (49.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_ngrok import run_with_ngrok
# app = Flask(__name__)
app = Flask(__name__, template_folder='/content/drive/MyDrive/Colab Notebooks/Web/templates')
run_with_ngrok(app)

@app.route("/", methods=['GET','POST'])
def start():
  result = None
  input_data = ''
  if request.method == 'POST':
    # 사용자가 입력한 자바 소스코드
    input_data = request.form['input_data']

    # 입력한 소스코드로 java 파일 생성
    f = open("/content/drive/MyDrive/jongsul/input.java","w+")
    f.write(input_data)
    f.close()

    # cpg 생성
    script_path = '/content/drive/MyDrive/jongsul/fixed.sh'
    execute_script(script_path)

    print("cpg 생성됨")

    # csv 생성
    folder_path = "/content/drive/MyDrive/jongsul/output"
    output_csv_path = "/content/drive/MyDrive/jongsul/csv/toCsv.csv"

    print("csv 생성됨")


    # Create or clear the existing CSV file and write the header
    write_to_csv_header(output_csv_path)

    process_all_dot_files(folder_path, output_csv_path)


    # 예측
    # 예측에 사용할 데이터 불러오기
    predict_dataset = CPGPredictDataset(csv_file='/content/drive/MyDrive/jongsul/csv/toCsv.csv')  # 예측에 사용할 데이터의 경로
    predict_data_list = [predict_dataset[i] for i in range(len(predict_dataset.graph_list))]

    # DataLoader 생성
    predict_loader = DataLoader(predict_data_list, batch_size=64)

    # 모델 불러오기
    model = GCN(num_node_features=num_node_features)
    model.load_state_dict(torch.load('/content/drive/MyDrive/jongsul/gcn_model.pth'))
    model = model.to(device)
    model.eval()

    print("모델 불러옴")


    # 예측 수행
    predictions = []
    with torch.no_grad():
        for predict_data in predict_loader:
            print("예측")
            predict_data = predict_data.to(device)
            output = model(predict_data)
            predictions.extend(output.cpu().numpy())

    print("예측 수행함")

    # 예측 결과 출력
    sum = 0
    count = 0
    for i, prediction in enumerate(predictions):
        print(str(i)+"번째 예측 : "+str(interpret_prediction(prediction)))
        sum+=interpret_prediction(prediction)
        count+=1
        # print(f"Time Complexity of input data is {interpret_prediction(prediction)}")  # 예측 값을 해석하여 출력합니다.

    resVal = sum/count

    result = ""

    if 0 <= resVal <= 0.5:
      result = "O(1)"
    elif 0.5 < resVal <= 1.5:
      result = "O(logn)"
    elif 1.5 < resVal <= 2.5:
      result = "O(n)"
    elif 2.5< resVal <= 3.5:
      result = "O(nlogn)"
    elif 3.5< resVal <= 4.5:
      result = "O(n^2)"
    else:
      result = "bigger"

    print("result 세팅함")

    # 결과
    os.system('rm -rf /content/drive/MyDrive/jongsul/output')
    os.system('rm -rf /content/drive/MyDrive/jongsul/csv/toCsv.csv')
    os.system('rm -rf /content/drive/MyDrive/jongsul/input.java')


  return render_template("index.html", input_data=input_data, result=result)
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://4036-35-237-20-254.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 17:17:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 17:17:59] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 17:18:17] "POST / HTTP/1.1" 200 -


cpg 생성됨
csv 생성됨
Processing file: 0-cpg.dot
Processing file: 1-cpg.dot
Processing file: 2-cpg.dot
Processing file: 3-cpg.dot
Processing file: 4-cpg.dot
Processing file: 5-cpg.dot
Processing file: 6-cpg.dot
Convert "CPG"csv to graph
Complete!
모델 불러옴
예측
예측 수행함
0번째 예측 : 1.218
1번째 예측 : 1.291
2번째 예측 : 1.301
3번째 예측 : 1.301
4번째 예측 : 1.288
5번째 예측 : 1.31
6번째 예측 : 1.301
result 세팅함


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 17:19:20] "POST / HTTP/1.1" 200 -


cpg 생성됨
csv 생성됨
Processing file: 0-cpg.dot
Processing file: 1-cpg.dot
Processing file: 2-cpg.dot
Processing file: 3-cpg.dot
Processing file: 4-cpg.dot
Processing file: 5-cpg.dot
Processing file: 6-cpg.dot
Convert "CPG"csv to graph
Complete!
모델 불러옴
예측
예측 수행함
0번째 예측 : 1.205
1번째 예측 : 1.291
2번째 예측 : 1.301
3번째 예측 : 1.301
4번째 예측 : 1.288
5번째 예측 : 1.31
6번째 예측 : 1.301
result 세팅함


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 17:20:44] "POST / HTTP/1.1" 200 -


cpg 생성됨
csv 생성됨
Processing file: 0-cpg.dot
Processing file: 1-cpg.dot
Processing file: 2-cpg.dot
Processing file: 3-cpg.dot
Convert "CPG"csv to graph
Complete!
모델 불러옴
예측
예측 수행함
0번째 예측 : 1.234
1번째 예측 : 1.25
2번째 예측 : 1.266
3번째 예측 : 1.259
result 세팅함


In [ ]:

os.system('rm -rf /content/drive/MyDrive/jongsul/output')
os.system('rm -rf /content/drive/MyDrive/jongsul/csv/toCsv.csv')
os.system('rm -rf /content/drive/MyDrive/jongsul/input.java')


0